In [2]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/home/tigranerfaceb/image-to-tex-OCR/Jupyter_Notebooks
Current path:/home/tigranerfaceb/image-to-tex-OCR


In [3]:
# Import the Printed_Tex_Data_Module

from Data.Handwritten_tex_experiment import Handwritten_Data_Module
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
from IPython.display import display, Math
from Models.Printed_Tex_Transformer import ResNetTransformer
from Lightning_Models.Printed_Tex_Lit_Model import LitResNetTransformer
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import transforms

In [4]:
'''
path_to_formulas = 'Data/Data_Bank/formulas.lst',
                  path_to_image_names= 'Data/Data_Bank/images',


'''


dataset = Handwritten_Data_Module(stage = 'fit',
                 set_max_label_length = 256,
                 number_png_images_to_use_in_dataset=200*1000,
                 labels_transform='default',
                image_transform_name ='alb',
                 train_val_fraction = 0.98,
                
                load_vocabulary = False,
                 


                 batch_size = 64,
                num_workers = 10,
                data_on_gpu = False,
                )



# Generates the dataframe from the images and formulas
# images need to be in the folder Data/Data_Bank/generated_png_images
# formulas need to be in Data/Data_Bank/final_png_formulas.txt
# image filenames need to be in Data/Data_Bank/corresponding_png_images.txt
dataset.prepare_data()

# if one wants to load a vocabulary do not use data.prepare_dataframe(), 
# but instead set load_vocabulary = True


In [5]:
dataset.setup()

Train/Val Data is ready for Model loading.


In [6]:
print( 
    'Vocabulary size:',len(dataset.vocabulary),
    "\nTrain-Dataset size:", len(dataset.data_train),
    "\nValidation-Dataset size:", len(dataset.data_val),
    '\nMax label length:', dataset.max_label_length,
    "\nStart <S> goes to index ",dataset.vocabulary['<S>'],
      "\nEnd <E> goes to index ",dataset.vocabulary['<E>'],
      "\nPadding <P> goes to index ",dataset.vocabulary['<P>'],
)

Vocabulary size: 550 
Train-Dataset size: 77480 
Validation-Dataset size: 1582 
Max label length: 258 
Start <S> goes to index  0 
End <E> goes to index  1 
Padding <P> goes to index  2


In [7]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
# Initiate the model
model = ResNetTransformer(dataset=dataset).to(dev)

/home/tigranerfaceb/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tigranerfaceb/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
# Load the weights

model.load_state_dict(torch.load(("Models_Parameters_Log/258_Test_run.pth"), map_location=torch.device('cpu')))


<All keys matched successfully>

In [10]:
%load_ext tensorboard 
%tensorboard --logdir Models_Parameters_Log/

In [11]:
logger = TensorBoardLogger("Models_Parameters_Log", name="Test_Run_HW")
lit_model = LitResNetTransformer(model=model)

#use the lit_model to run with Trainer
trainer = Trainer(accelerator='gpu', devices=1,  max_epochs=60, logger=logger, auto_lr_find=True) #accumulate_grad_batches=2, overfit_batches=1, default_root_dir="lightning_logs/ResNet/checkpoints"  
trainer.fit(model=lit_model, datamodule=dataset,ckpt_path='Models_Parameters_Log/Test_Run/version_3/checkpoints/epoch=37-step=53466.ckpt')


wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


/home/tigranerfaceb/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


MisconfigurationException: No supported gpu backend found!

In [ ]:
# save model
torch.save(model.state_dict(), 'Models_Parameters_Log/258_Test_HW.pth')

# save vocabulary
# save_dic(dataset.vocabulary)